In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Hierarchical indexing (MultiIndex)

In [ ]:
A = pd.DataFrame({'col1': ['a', 'a', 'a', 'b', 'b', 'b', 'c', 'c', 'c'],
                  'col2': ['d', 'e', 'f', 'd', 'e', 'f', 'd', 'e', 'f'],
                  'col3': [1, 2, 3, 4, 5, 6, 7, 8, 9]})
A

In [ ]:
B = A.set_index(['col1', 'col2'])
B

In [ ]:
# indexing is similar to regular index
B.loc['a']

In [ ]:
# tuple to index an item
B.loc[('b', 'e')]

In [ ]:
# slicing
B.loc['b':'c']

In [ ]:
# to check out all 'f'
B.loc[B.index.get_level_values('col2') == 'f']

In [ ]:
# or your can use index slicer
B.loc[pd.IndexSlice[:, 'f'], :]

In [ ]:
# or you can use cross-section method xs
B.xs('f', level=1, drop_level=False)

In [ ]:
# to restore index
B.reset_index()

# Reshaping and pivot tables

In [ ]:
A

In [ ]:
B = A.pivot(index='col1', columns='col2')
B

In [ ]:
B.stack()

In [ ]:
users = pd.read_csv('users.zip', sep='|')
users

In [ ]:
users.pivot_table(values='age', index='occupation', columns='gender', aggfunc=['min', 'max', 'mean', 'std'])

In [ ]:
# a very useful function to categorize numbers
pd.cut(users.age, [0, 20, 40, 60, 100])

In [ ]:
# it is effortless to calculate the female vs male ratio of different age groups
users['age_group'] = pd.cut(users.age, [0, 30, 100])
gender_age = users.pivot_table(values='age', index='occupation', columns=['gender', 'age_group'], aggfunc=['count'])
gender_age

In [ ]:
gender_age.plot.bar(figsize=(13, 6), stacked=True)

# Groupby

## Aggregation

In [ ]:
# calculate female max age for each occupation
users.query('gender == "F"').groupby('occupation').age.max()

In [ ]:
# applying to specific columns
users.groupby('gender').agg({'age':np.mean, 'user_id':np.size})

In [ ]:
# applying multiple functions
users.groupby('gender').age.agg(m=np.mean, s=np.size)

In [ ]:
# Week 4 exercise
Brand = ['Ford', 'Ford',   'Chevy',   'Chevy',   'Honda', 'Ford',   'Honda', 'Honda', 'Ford', 'Chevy']
Model = ['F150', 'Escape', 'Charger', 'Charger', 'Civic', 'Escape', 'CRV',   'CRV',   'F150', 'Silverado']
Type = ['Pickup', 'SUV', 'Sedan', 'Sedan', 'Sedan', 'SUV', 'SUV',   'SUV',   'Pickup', 'Pickup']
Accidents = [25, 79, 46, 90, 29, 88, 79, 93, 20, 11]
cars = pd.DataFrame(dict(brand=Brand, model=Model, type=Type, accidents=Accidents))
cars

In [ ]:
# Find the brand and model of the safest car
cars.groupby(['brand', 'model']).accidents.mean().sort_values().tail(3)

In [ ]:
# Find the safest brand for each vehicle type (one line)

cars.groupby('brand').accidents.agg(s=np.sum, m=np.mean)

In [ ]:
# Calculate the total accidents for each brand and vehicle type, respectively.




# Time Series

## Convert to Timestamp

In [ ]:
# You can use either pd.to_datetime or pd.Timestamp
print(pd.to_datetime('2010/11/12'))
print(pd.Timestamp('2020/11/12'))

In [ ]:
# pandas usually is very intelligent to convert dates in different format but you can also specify the format mannually
print(pd.to_datetime('2010/11/12', format='%Y/%m/%d'))
print(pd.to_datetime('12-11-2020 00:00', format='%d-%m-%Y %H:%M'))

## Construct DatatimeIndex

In [ ]:
# pd.DatetimeIndex is frequently used to construct datetimeIndex
pd.DatetimeIndex(['2005/11/23', '2010.12.31', 'Jul 31, 2009', '2012-2-28'])

In [ ]:
#  to_datetime can also be used
pd.to_datetime(['2005/11/23', '2010.12.31', 'Jul 31, 2009', '2012-2-28'])

In [ ]:
#  to_datetime will retuen series if the input is Series
pd.to_datetime(pd.Series(['2005/11/23', '2010.12.31', 'Jul 31, 2009', '2012-2-28']))

In [ ]:
# but if you use it as index for Series or DataFrame it will be converted to DatatimeIndex automatically
A = pd.Series(range(4), index=pd.to_datetime(pd.Series(['2005/11/23', '2010.12.31', 'Jul 31, 2009', '2012-2-28'])))
A.index

In [ ]:
# if you have a DataFrame with year, month and day columns, you could convert to TimeStamp Series
df = pd.DataFrame({'year': [2015, 2016], 'month': [2, 3], 'day': [4, 5], 'hour': [2, 3]})
print(df, '\n\n')

print(pd.to_datetime(df))

## Generating ranges of timestamps

In [ ]:
start = pd.datetime(2011, 1, 1)
end = pd.datetime(2012, 12, 31)
time_idx = pd.date_range(start, end, freq='M')
time_idx

In [ ]:
# if you just want to see the business days
pd.bdate_range(start, end, freq='M')

## Indexing Datetime

In [ ]:
time_idx = pd.date_range('2000', '2009', freq='D')
time_ser = pd.Series(np.random.random(time_idx.shape), index=time_idx)
time_ser

In [ ]:
#  If the string is less accurate than the index, it will be treated as a slice, otherwise as an exact match.
print('2005-01-01\n', time_ser['2005-01-01'], '\n\n')
print('2005-01\n', time_ser['2005-01'])

In [ ]:
# time indexing
print(time_ser['2005-01-01':'2007-02-15'])


In [ ]:
# partial indexing
time_ser['2005-01':'2007-02']

In [ ]:
# partial indexing, year only
time_ser['2005':'2007']

In [ ]:
# Shifting / lagging
time_ser.shift(2, freq='W') # careful this will also change the frequency of the index

## Resampling

In [ ]:
# basic
rs = time_ser.resample('M')
rs

In [ ]:
# apply stat functions: sum, mean, std, sem, max, min, median, first, last, ohlc:
print('monthly sum:\n', time_ser.resample('M').sum(), '\n\n')
print('monthly mean:\n', time_ser.resample('MS').mean())

In [ ]:
# apply general function
time_ser.resample('M').apply(lambda x: (x**2).sum())

# Visualization

In [ ]:
users.groupby('occupation').age.mean().plot()

In [ ]:
users.groupby('occupation').age.mean().plot.bar()

In [ ]:
tractor = pd.read_excel('test.10-24.2-time_RAR.xlsx', 'test.10-24.2-time')
tractor

In [ ]:
tractor.plot.scatter('longitude', 'lattitude', s=1, c='eng_torque', cmap='jet', vmin=10, vmax=15)

In [ ]:
# seems there is outlier
tractor_clean = tractor.query('lattitude > 40')
tractor_clean.plot.scatter('longitude', 'lattitude', s=1, c='eng_torque', cmap='jet', vmin=10, vmax=15)

# Covid19 Analysis

In [ ]:
#
covid19 = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
covid19
# change the column names
covid19.rename({'Province/State':'Province', 'Country/Region':'Country'})
# exclude mainland china because the outbreak has occured and has been contained
covid19_nochina = covid19.query('Country/Region != "Mainland China"')
covid19_nochina